# Predicting Ethiopian Vehicle Insurance Premiums

- The goal of this project is to clean, analyse and predict vehicle insurance premiums of the state-owned Ethiopian Insurance Corporation (one of the biggest insurance companies in Ethiopia).
- The dataset we'll use describes vehicles, their insurance premiums and other insurance related atributes from July 2011 to June 2018. It can be found on [Mendeley Data](https://data.mendeley.com/datasets/34nfrk36dt/1).



### Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, r2_score

from xgboost import XGBRegressor, plot_importance
from scipy.stats import expon, kstest
from fuzzywuzzy import process

from skopt import BayesSearchCV
from skopt.space import Real, Integer

insurance_data_1 = pd.read_csv("insuranceData/motor_data11-14lats.csv")
insurance_data_2 = pd.read_csv("insuranceData/motor_data14-2018.csv")

## Predefined function 

In [ ]:
def plot_counts_and_premiums(ax, count_series, premium_series, xlabel):
    index = count_series.index
    x = np.arange(len(index))
    w = 0.4

    ax.bar(x - w/2, count_series.values, w, label="Policy Count", color="tab:red")
    ax.set_ylabel("Policy Count", color="tab:red")
    ax.tick_params(axis="y", labelcolor="tab:red")
    ax.set_xlabel(xlabel)
    ax.set_xticks(x)
    ax.set_xticklabels(index)

    ax2 = ax.twinx()
    ax2.bar(x + w/2, premium_series.loc[index].values, w, label="Premium Sum", color="tab:blue")
    ax2.set_ylabel("Premium Sum", color="tab:blue")
    ax2.tick_params(axis="y", labelcolor="tab:blue")

## Dataset overview

Both of the provided dataset files include the same entry attributes and differ only in entry dates. They will need to be merged.

In [ ]:
insurance_data = pd.concat([insurance_data_1, insurance_data_2], ignore_index=True)

print(insurance_data.shape)

We have 802036 insurance policy records and 16 policy related attributes.

Now let's look whether the provided attributes have been read correctly.

In [ ]:
insurance_data.head(10)

There are multiple entries regarding the same vehicle as it has to be reinsured at least every year. That can lead to up to 7 entries for the same vehicle with only the premium amout fluctuating.

In [ ]:
insurance_data.info()

Columns seem to have been read correctly. Let's now look at the values in individual columns.

### Object ID

In [ ]:
insurance_data["OBJECT_ID"].value_counts()

Some vehicles appear more that 7 times in the dataset. This can happen when the owner of a vehicle changes more than once per year.

In [ ]:
insurance_data[insurance_data["OBJECT_ID"].astype(str) == "5000116673"]

In [ ]:
(insurance_data["OBJECT_ID"].value_counts() > 7).sum()

There are a total of 7290 object IDs that apprear more than 7 times.

It quickly becomes clear that we are dealing with a dataset with many errors. For the same object ID, there are entries that are not clearly logically intertwined. However, we can still perform predictive analysis under the presumption that each row in the dataset represents a unique vehicle.

### Sex

In [ ]:
insurance_data["SEX"].value_counts()

In the dataset, there are 3 unique sex values with 0 being legal entities, 1 - males and 2 - females. The number of insurance contracts in which men are the policyholders is 4.67 times greater that the number of contracts with female policyholders. That is due to women being less likely to have a drivers license in Ethiopia.

Let's remap the value for better clarity.

In [ ]:
sex_mapping = {0:"LEGAL ENTITY", 1:"MALE", 2:"FEMALE"}

insurance_data["SEX"] = insurance_data["SEX"].map(sex_mapping)

insurance_data["SEX"].value_counts()

In [ ]:
insurance_data["LOG_PREMIUM"] = np.log(insurance_data["PREMIUM"] + 1)

sex_values = insurance_data["SEX"].unique()

plt.figure(figsize=(8, 5))

for sex in sex_values:
    subset = insurance_data[insurance_data["SEX"] == sex]["LOG_PREMIUM"]
    plt.hist(subset, bins=30, alpha=0.5, label=str(sex), density=True)

plt.xlabel("Log Premium")
plt.ylabel("Proportion")
plt.legend()
plt.show()

The distributions of premiums for males and females are practically inseparable. For legal entities, however, the premiums seem to be larger.

### Insurance start & end date

In [ ]:
insurance_data["INSR_BEGIN"].value_counts()

In [ ]:
insurance_data["INSR_END"].value_counts()

There do not seem to be any obvious errors in the data. Yet, we must check whether there are entries where the insurance start date is later than the end date.

In [ ]:
insurance_data["INSR_BEGIN"] = pd.to_datetime(insurance_data["INSR_BEGIN"], format="%d-%b-%y")

insurance_data["INSR_END"] = pd.to_datetime(insurance_data["INSR_END"], format="%d-%b-%y")

end_greater_start = insurance_data["INSR_BEGIN"] > insurance_data["INSR_END"]
length = len(end_greater_start[end_greater_start == True])
length

No end values are earlier than start values. We can now visualize the variable data.

In [ ]:
insurance_data["INSR_START_MONTH"] = pd.to_datetime(insurance_data["INSR_BEGIN"], format="%d-%b-%y").dt.month
insurance_data["INSR_START_YEAR"] = pd.to_datetime(insurance_data["INSR_BEGIN"], format="%d-%b-%y").dt.year

start_months = insurance_data["INSR_START_MONTH"].value_counts().sort_index()
start_months_premium = insurance_data.groupby("INSR_START_MONTH")["PREMIUM"].sum()

start_years = insurance_data["INSR_START_YEAR"].value_counts().sort_index()
start_years_premium = insurance_data.groupby("INSR_START_YEAR")["PREMIUM"].sum()

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

plot_counts_and_premiums(axes[0], start_months, start_months_premium, "Month")
plot_counts_and_premiums(axes[1], start_years, start_years_premium, "Year")

fig.tight_layout()
plt.show()

From the charts it becomes clear that policy count closely correlates with premium sums with regard to both the month and the year of the insurance start date. One month, July, stand out as the month in which the most policies are introduced. Also, January and December seem to be months where more policies are introduced than usual. This could be due to accounting methods in which a policy signed near the end of the fiscal year is counted as being signed in the next year. In terms of the trend regarding the year, there is a clear pattern of growth from 2011 to 2017 and a sharp drop off of new policies in 2018. The drop off can be explained by the end of data collection period being June of 2018. We can check whether growth in the month of June in the respective years is equally as rapid.

In [ ]:
before_june = insurance_data[(insurance_data["INSR_START_YEAR"].isin([2017, 2018])) & (insurance_data["INSR_START_MONTH"] < 7)]

before_june.groupby("INSR_START_YEAR").size().reset_index(name="Policy Count")

The pace of growth is about the same.

Let"s also create a dummy variable for insurance start days.

In [ ]:
insurance_data["INSR_START_DAY"] = pd.to_datetime(insurance_data["INSR_BEGIN"], format="%d-%b-%y").dt.day

In [ ]:
day_stats = insurance_data.groupby("INSR_START_DAY").agg(
    policy_count=("PREMIUM", "count"),
    premium_sum=("PREMIUM", "sum")
)

fig, ax1 = plt.subplots(figsize=(12, 6))

day_stats["policy_count"].plot(kind="bar", ax=ax1, position=0, width=0.4, label="Policy Count")

ax2 = ax1.twinx()
day_stats["premium_sum"].plot(kind="bar", color="red", ax=ax2, position=1, width=0.4, label="Premium Sum")

ax1.set_xlabel("Day")   
ax1.set_ylabel("Number of Policies")
ax2.set_ylabel("Total Premium Sum", color="red")
plt.title("Policies and Premium Sums by Start Day")
plt.tight_layout()

plt.show()

The largest of policies were registered on the 1st and the 8th. This is abnormal, and the distribution should be uniform. This issue should be escalated to the data owners.

### Effective year

In [ ]:
insurance_data["EFFECTIVE_YR"].value_counts()

The effective year variable indicates what year the policy came into effect (was first insured with the company). There are numerous records that indicate a year before the historic start date of the dataset (2011).

Yet, the column contains values that are not indicative of a number and should be removed. Since there are a total of 802036 records, we can afford to lose quite a few. We also need to convert the years into a four-digit number as a two-digit year encoding is only common on legacy data storage systems.

In [ ]:
insurance_data = insurance_data[insurance_data["EFFECTIVE_YR"].astype(str).str.match(r"^\d{2}$")]

def convert_year(y):
    y = int(y)
    if y > 18:
        return 1900 + y
    else:
        return 2000 + y

insurance_data["EFFECTIVE_YR_FULL"] = insurance_data["EFFECTIVE_YR"].apply(convert_year)

insurance_data.shape

After cleaning the effective year column, we have lost 1171 rows.

In [ ]:
year_counts = insurance_data["EFFECTIVE_YR_FULL"].value_counts().sort_index()

year_counts = year_counts[
    (year_counts.index >= 1992) &
    (year_counts.index <= 2018)
]

year_counts.plot(kind="bar")
plt.xlabel("Effective Year")
plt.ylabel("Policy Count")
plt.xticks(rotation=45)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
year_counts = insurance_data["EFFECTIVE_YR_FULL"].value_counts().sort_index()
print(year_counts)

The data shows that most of the insured vehicles were first insured in 2011 and later. After contacting the postdoctoral fellow that published the dataset, I was informed that the meaning of this variable is not very well documented. There are values of 1947 and prior, eventhough, in Ethiopia, the first motor insurance was issued in 1947. This variable will be dropped.

In [ ]:
insurance_data.drop(columns=["EFFECTIVE_YR", "EFFECTIVE_YR_FULL"], inplace=True)

### Insurance type

In [ ]:
insurance_data["INSR_TYPE"].value_counts()

There are a total of three types of insurance: 1201 - private, 1202 - commercial and 1204 - motor trade road risk (for motor trade workers that drive vehicles they do not personally own, such as mechanics when testing repaired vehicles).

Let's change the values so they make more sense.

In [ ]:
insurance_type_mapping = {1202:"PRIVATE", 1201:"COMMERCIAL", 1204:"MOTOR TRADE"}

insurance_data["INSR_TYPE"] = insurance_data["INSR_TYPE"].map(insurance_type_mapping)

In [ ]:
insurance_data["INSR_TYPE"].value_counts()

In [ ]:
insurance_type_values = insurance_data["INSR_TYPE"].unique()

plt.figure(figsize=(8, 5))

for type in insurance_type_values:
    subset = insurance_data[insurance_data["INSR_TYPE"] == type]["LOG_PREMIUM"]
    plt.hist(subset, bins=30, alpha=0.5, label=str(type), density=True)

plt.xlabel("Log Premium")
plt.ylabel("Proportion")
plt.legend()
plt.show()

Here, motor trade insurance premiums are not as high as private and commercial premiums. Commercial and private insurance premiums are distributed around the same. It should also be noted that the log premium value range is similar for later mentioned groups.


### Insured value

In [ ]:
insurance_data["INSURED_VALUE"].value_counts()

343235 vehicles in the dataset have no provided insurance value. Insured value of 0 means the policyholder has the liability insurance coverage only, not the comprehensive coverage while insured value higher than 0 indicates comprehensive coverage.

The difference between the liability and comprehensive insurance is that liability insurance covers damage or injury you cause to other people or their property (and not repairs to your vehicle) and comprehensive insurance covers non-collision damage to your own car.

For the purpose of developing a model, we may create an additional variable that indicates the type of insurance the policyholder has.

In [ ]:
insurance_data["INSR_COVER"] = np.where(
    insurance_data["INSURED_VALUE"] == 0,
    "LIABILITY",
    "COMPREHENSIVE"
)

insurance_data["INSR_COVER"].value_counts()

In [ ]:
insurance_data["INSURED_VALUE"].describe()

In [ ]:
plt.figure(figsize=(8, 5))
plt.scatter(insurance_data["INSURED_VALUE"], insurance_data["PREMIUM"], alpha=0.2)
plt.xlabel("Insured Value")
plt.ylabel("Premium")
plt.xlim([-2000000, 50000000])
plt.ylim([-20000, 400000])
plt.show()

It is unlikely that there is a strong linear relation between insured value and premiums.

### Year of production

In [ ]:
insurance_data["PROD_YEAR"].value_counts()

In [ ]:
insurance_data["PROD_YEAR"].unique()

We have NaN values in our dataset. Instead of removing them, we can fill them with median values and preserve the central tendency.

In [ ]:
insurance_data["PROD_YEAR"] = insurance_data["PROD_YEAR"].fillna(insurance_data["PROD_YEAR"].median())

In [ ]:
insurance_data["PROD_YEAR"].describe()

In [ ]:
insurance_data["PROD_YEAR"].plot(kind="hist", bins=40)

plt.show()

The production year variable is heavily left-skewed, but are no abnormalities in this attribute. We can create an additional variable for vehicle age.

In [ ]:
max_insr_date = pd.to_datetime(insurance_data["INSR_BEGIN"].max())

prod_year_dt = pd.to_datetime(insurance_data["PROD_YEAR"], format="%Y")

insurance_data["VEHICLE_AGE"] = (max_insr_date - prod_year_dt).dt.days / 365

In [ ]:
plt.figure(figsize=(8, 5))
plt.scatter(insurance_data["VEHICLE_AGE"], insurance_data["PREMIUM"], alpha=0.1)
plt.xlabel("Vehicle Age")
plt.ylabel("Premium")
plt.ylim([-20000, 300000])
plt.show()

Premiums tend to stabilize and decrease as vehicles age. There are likely few antique vehicles in Ethiopia, so practically all vehicles become less valuable with age.

### Number of seats

In [ ]:
insurance_data["SEATS_NUM"].describe()

In [ ]:
insurance_data["SEATS_NUM"].value_counts()

There are a total of 59703 vehicles with 0 seats which is impossible. Also, the number of seats should not exceed 256 (seats in the largest bus in the world). Other values will be considered correct.

In [ ]:
insurance_data = insurance_data[(insurance_data["SEATS_NUM"] > 0) & (insurance_data["SEATS_NUM"] <= 256)]

print(insurance_data.shape)

### Carrying capacity

In [ ]:
insurance_data["CARRYING_CAPACITY"].value_counts()

It is clear that the seat number variable and the carrying capacity variable are not clearly differentiated. They have been mixed up and should be removed.

In [ ]:
insurance_data.drop(columns=["SEATS_NUM", "CARRYING_CAPACITY"], inplace=True)

### Vehicle type

In [ ]:
insurance_data["TYPE_VEHICLE"].value_counts()

Nothing out of the ordinary here. We will capitalize the letters for consistency.

In [ ]:
insurance_data["TYPE_VEHICLE"] = insurance_data["TYPE_VEHICLE"].str.upper()

### Vehicle Weight

In [ ]:
insurance_data["CCM_TON"].describe()

In [ ]:
insurance_data["CCM_TON"].value_counts()

This variable is related to the weight of the vehicle. Since it is not clearly described how and what units are being used, we will remove it.

In [ ]:
insurance_data.drop(columns=["CCM_TON"], inplace=True)

### Vehicle maker

In [ ]:
insurance_data["MAKE"].value_counts()

In [ ]:
insurance_data["MAKE"].value_counts()[insurance_data["MAKE"].value_counts() <= 200]

There is a large number of vehicle maker names that are not representative of the manufacturer or are miss-spellings of the brand name, with 'TOYOTA MERCHEDIS' being the most humorous one. Since there is a total number of 657 unique brands, individual corrections would be too cumbersome. Instead, we can use a list of car names and check for matches in the dataset.

We will scrape a complete list of car brands from a car brand [website](https://www.carlogos.org/).

In [ ]:
url = "https://www.carlogos.org/car-brands-a-z/"

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/115.0.0.0 Safari/537.36"
    )
}

response = requests.get(url, headers=headers)
response.raise_for_status()

soup = BeautifulSoup(response.text, "html.parser")

brands = []

for dd in soup.find_all("dd"):
    a_tag = dd.find("a")
    if a_tag and a_tag.text.strip():
        brands.append(a_tag.text.strip())

brands = list(dict.fromkeys(brands))

print(brands)

len(brands)

The website states that there are 383 car brands on the website. The beginning of the list contains countries and the end contains other unrelated text. That should prove easy to clean.

In [ ]:
brands_clean = brands[21:-8]

remove_list = ["Mercedes-Benz", "Audi Sport", "BMW M", "Chevrolet Corvette", "Ford Mustang", "Nissan GT-R", "Toyota Alphard", "Toyota Century", "Toyota Crown"]
brands_clean = [b for b in brands_clean if b not in remove_list]

brands_clean = ["Mercedes" if b == "Mercedes-AMG" else b for b in brands_clean]

print(brands_clean)

len(brands_clean)

We can now look for matching brands in our dataset. For that we will use the fuzzywuzzy library and search for matches using the Levenshtein distance.

In [ ]:
insurance_data = insurance_data[insurance_data["MAKE"].str.strip() != "*"]

In [ ]:
unique_makes = insurance_data["MAKE"].unique()

def match_brand(brand):
    match, score = process.extractOne(brand, brands_clean)
    return match if score >= 80 else "UNKNOWN"

mapping = {make: match_brand(make) for make in unique_makes}

We can review and map the remaining brands that were not detected as matches manually.

In [ ]:
manual_mapping = {
    "MERCEEDICE": "Mercedes",
    "MERCEEDES": "Mercedes",
    "MERCHEDES": "Mercedes",
    "MERCEDICE": "Mercedes",
    "MERCEDIS": "Mercedes",
    "DUNGFING": "Dongfeng",
    "GEEP": "Jeep",
    "PEAGOUT": "Peugeot",
    "PAGOT": "Peugeot",
    "PEJOT": "Peugeot",
    "LAND CRUISER": "Toyota",
    "T0Y0TA": "Toyota",
    "COROLLA": "Toyota",
    "RAV4": "Toyota",
    "VOLSVAGON": "Volkswagen",
    "PASSAT": "Volkswagen",
    "HYUNDI GETZ": "Hyundai",
    "DISCOVER": "Land Rover",
    "FOED": "Ford",
    "BMB": "BMW"
}

combined_mapping = {**mapping, **manual_mapping}

insurance_data["MANUFACTURER"] = insurance_data["MAKE"].map(mapping)

In [ ]:
insurance_data["MANUFACTURER"].unique()

### Primary function of vehicle

In [ ]:
insurance_data["USAGE"].value_counts()

All functions seem valid.

In [ ]:
insurance_data["USAGE"] = insurance_data["USAGE"].str.upper()

### Paid claim sum

In [ ]:
insurance_data["CLAIM_PAID"] = insurance_data["CLAIM_PAID"].replace(np.nan, 0)

pd.set_option("display.float_format", "{:.2f}".format)

print(insurance_data["CLAIM_PAID"].describe())

In [ ]:
log_claim_paid = np.log(insurance_data["CLAIM_PAID"] + 1)

log_claim_paid.plot(kind="hist")

plt.show()

In [ ]:
len(insurance_data[insurance_data["CLAIM_PAID"] == 0])

The payout values seem realistic since they are in Ethiopian birr. The largest paid sum is around 1,100,000 USD.

There is one issue: even the log-transformed variable does not resemble any known distribution. Thus, we can create an additional variable that might perform better in the model.

In [ ]:
insurance_data["WAS_CLAIM_PAID"] = np.where(insurance_data['CLAIM_PAID'] > 0, True, False)

insurance_data["WAS_CLAIM_PAID"].value_counts()

### Insurance premium

The premium amounts are provided in Ethiopian birr (1000 Birr = 7,3 USD).

In [ ]:
insurance_data["PREMIUM"][insurance_data["PREMIUM"] <= 0].count()

There are 10 vehicles with a premium of 0 or less. This is not acceptable and we will remove them. I was informed by the publisher of the dataset that the company follows a principle of "No premium, no insurance", meaning they are no longer insured. This method of record keeping is ineffective.

In [ ]:
insurance_data = insurance_data[insurance_data["PREMIUM"] > 0]

insurance_data["PREMIUM"].describe()

From the variable description it becomes clear that it is nowhere close to normality.

In [ ]:
params = expon.fit(insurance_data["PREMIUM"])

x = np.linspace(0, 50000, 1000)
pdf = expon.pdf(x, *params)

plt.hist(insurance_data["PREMIUM"], bins=500, density=True)
plt.plot(x, pdf, 'r')
plt.xlim([0, 50000])
plt.show()

The distribution closely resembles an exponential one.

In [ ]:
D, p_value = kstest(insurance_data["PREMIUM"], "expon", args=params)

print(f"P-value: {p_value:.4f}")

As the p-value is less than 0.05, we fail to reject the null hypothesis. This distribution might as well exponential.

In [ ]:
log_premiums = np.log(insurance_data["PREMIUM"])

fig, axes = plt.subplots(2, 1, figsize=(8, 6), gridspec_kw={"height_ratios": [1, 4]})

axes[0].boxplot(log_premiums, vert=False, widths=0.6)
axes[0].set_xticks([])
axes[0].set_yticks([])

axes[1].hist(log_premiums, bins=25)
axes[1].set_xlabel("Log Premium")

plt.tight_layout(h_pad=0)

plt.show()

### Additional variables for modelling

From the existing dataset variable we can crate additional variables that, in theory, could provide better model outcomes.

#### Previous claim paid

If a vehicle is dangerous to operate this could be reflected in the total claim payout sum.

In [ ]:
sorted_df = insurance_data.sort_values(["OBJECT_ID", "INSR_BEGIN"])

sorted_df["PREVIOUS_CLAIM_PAID"] = (
    sorted_df.groupby("OBJECT_ID")["CLAIM_PAID"]
    .cumsum()
    .shift(fill_value=0)
)

insurance_data["PREVIOUS_CLAIM_PAID"] = sorted_df["PREVIOUS_CLAIM_PAID"].reindex(insurance_data.index)


#### Previous policy holders

A greater number of previous reinsurances could also indicate greater risk.

In [ ]:
sorted_df = insurance_data.sort_values(["OBJECT_ID", "INSR_BEGIN"])

sorted_df["PREVIOUS_POLICYHOLDERS"] = (
    sorted_df.groupby("OBJECT_ID").cumcount()
)

insurance_data["PREVIOUS_POLICYHOLDERS"] = sorted_df["PREVIOUS_POLICYHOLDERS"].reindex(insurance_data.index)

## Modelling

### Linear regression

In [ ]:
insurance_data.info()

We are left with 15 variables that could be used for modelling: `SEX`, `INSR_TYPE`, `INSURED_VALUE`, `PROD_YEAR`, `TYPE_VEHICLE`, `USAGE`, `CLAIM_PAID`, `INSR_COVER`, `INSR_START_DAY/MONTH/YEAR`, `INSR_COVER`, `VEHICLE_AGE`, `MANUFACTURER`, `WAS_CLAIM_PAID`, `PREVIOUS_CLAIM_PAID` and `PREVIOUS_POLICYHOLDERS`.

We will select variables by using R<sup>2</sup> as a guiding measure.

#### Model #1

In [ ]:
model1 = LinearRegression()

X = insurance_data[["SEX", "INSR_TYPE", "INSURED_VALUE", "PROD_YEAR", "TYPE_VEHICLE", "USAGE", "INSR_COVER", "CLAIM_PAID"]].copy()

X = pd.get_dummies(X, columns=["SEX", "INSR_TYPE", "PROD_YEAR", "TYPE_VEHICLE", "USAGE", "INSR_COVER"], drop_first=True)

Y = insurance_data["PREMIUM"]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=87)

model1.fit(X_train, Y_train)

Y_pred = model1.predict(X_test)

r_sq = model1.score(X_test, Y_test)

mae = mean_absolute_error(Y_test, Y_pred)

r_sq, mae

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

axes[0].scatter(Y_test, Y_pred, alpha=0.1)
axes[0].plot([Y_test.min(), Y_test.max()],
             [Y_test.min(), Y_test.max()],
             'r')
axes[0].set_title("Accuracy Plot")
axes[0].set_xlabel("Actual")
axes[0].set_ylabel("Predicted")

axes[1].scatter(Y_pred, Y_test - Y_pred, alpha=0.1)
axes[1].axhline(y=0, color='r')
axes[1].set_xlabel("Predicted")
axes[1].set_ylabel("Residuals")
axes[1].set_title("Residual Plot")

axes[2].hist(Y_test - Y_pred, bins=30, density=True)
axes[2].set_xlabel("Residuals")
axes[2].set_ylabel("Frequency")
axes[2].set_title("Distribution of Residuals")

plt.tight_layout()
plt.show()

In [ ]:
correlation = np.corrcoef(Y_test, Y_pred)[0, 1]
print(f"Correlation: {correlation:.2f}")

With almost all variables included in the model, we are getting an R<sup>2</sup> of 0.66. This is far from ideal, especially considering that we have not yet selected the best regressors.

#### Model #2

In [ ]:
model2 = LinearRegression()

X = insurance_data[["INSR_TYPE", "INSURED_VALUE", "TYPE_VEHICLE", "USAGE"]].copy()

X = pd.get_dummies(X, columns=["INSR_TYPE", "TYPE_VEHICLE", "USAGE"], drop_first=True)

Y = insurance_data["PREMIUM"]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=87)

model2.fit(X_train, Y_train)

Y_pred = model2.predict(X_test)

r_sq = model2.score(X_test, Y_test)

mae = mean_absolute_error(Y_test, Y_pred)

r_sq, mae

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

axes[0].scatter(Y_test, Y_pred, alpha=0.1)
axes[0].plot([Y_test.min(), Y_test.max()],
             [Y_test.min(), Y_test.max()],
             "r")
axes[0].set_title("Accuracy Plot")
axes[0].set_xlabel("Actual")
axes[0].set_ylabel("Predicted")

axes[1].scatter(Y_pred, Y_test - Y_pred, alpha=0.1)
axes[1].axhline(y=0, color="r")
axes[1].set_xlabel("Predicted")
axes[1].set_ylabel("Residuals")
axes[1].set_title("Residual Plot")

axes[2].hist(Y_test - Y_pred, bins=30, density=True)
axes[2].set_xlabel("Residuals")
axes[2].set_ylabel("Frequency")
axes[2].set_title("Distribution of Residuals")

plt.tight_layout()
plt.show()

After dropping the less influential variables through backward selection, we are left with 4 variables and an R<sup>2</sup> of 0.625. That greatly simplified our model while providing similar results, but it quickly becomes clear that linear regression is not the ideal model candidate for premium predictions. The distribution of errors is nowhere near normal and quite a few of linear regression assumptions are not met: linearity, homoscedasticity. The relationship between the regressors and premium values is not linear. Different models should be tested for their predictive ability.

### XGBoost

#### Model #3

Another model that we could test is XGBoost since it can handle non-linear relationships between variables and can potencially achieve better accuracy. We'll begin by adding all possible variables into the model.

In [ ]:
X = insurance_data[[
    "SEX", "INSR_TYPE", "INSURED_VALUE", "CLAIM_PAID",
    "INSR_COVER", "MANUFACTURER", "TYPE_VEHICLE", "USAGE",
    "PROD_YEAR", "INSR_START_MONTH", "INSR_START_YEAR",
    "INSR_START_DAY", "VEHICLE_AGE", "WAS_CLAIM_PAID",
    "PREVIOUS_CLAIM_PAID", "PREVIOUS_POLICYHOLDERS"
]].copy()

X["INSURED_VALUE"] = np.log1p(X["INSURED_VALUE"] + 1)
X["CLAIM_PAID"] = np.log1p(X["CLAIM_PAID"] + 1)

X = pd.get_dummies(
    X,
    columns=["SEX", "INSR_TYPE", "INSR_COVER", "MANUFACTURER", "TYPE_VEHICLE", "USAGE",
             "INSR_START_MONTH", "INSR_START_YEAR", "INSR_START_DAY",
             "WAS_CLAIM_PAID"],
    drop_first=True
)

Y = insurance_data["PREMIUM"]

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=87
)

model3 = XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=87,
    n_jobs=-1
)

model3.fit(X_train, Y_train)

Y_pred = model3.predict(X_test)

r_sq = r2_score(Y_test, Y_pred)
mae = mean_absolute_error(Y_test, Y_pred)

print(f"R²: {r_sq:.4f}")
print(f"MAE: {mae:.2f}")

plot_importance(model3, max_num_features=20, importance_type='weight')
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

axes[0].scatter(Y_test, Y_pred, alpha=0.1)
axes[0].plot([Y_test.min(), Y_test.max()],
             [Y_test.min(), Y_test.max()],
             "r")
axes[0].set_title("Accuracy Plot")
axes[0].set_xlabel("Actual")
axes[0].set_ylabel("Predicted")

axes[1].scatter(Y_pred, Y_test - Y_pred, alpha=0.1)
axes[1].axhline(y=0, color="r")
axes[1].set_xlabel("Predicted")
axes[1].set_ylabel("Residuals")
axes[1].set_title("Residual Plot")

axes[2].hist(Y_test - Y_pred, bins=30, density=True)
axes[2].set_xlabel("Residuals")
axes[2].set_ylabel("Frequency")
axes[2].set_title("Distribution of Residuals")

plt.tight_layout()
plt.show()

With only a model switch and all variables used we are getting an R<sup>2</sup> of 0.85. That is a great improvement over 0.625. Our accuracy plot data points are much more concentrated around the accuracy line. The residual plot still shows non-random concentration around the line and the spike around 0 in the residual distribution is quite normal as it shows that the model is accurate for most of the data.

#### Model #4

Still, there is room for improvement. We can remove that less meaningful variables from the feature importance chart and log-transform the premium values.

In [ ]:
X = insurance_data[[
    "SEX", "INSR_TYPE", "INSURED_VALUE",
    "INSR_COVER", "TYPE_VEHICLE", "PROD_YEAR",
    "VEHICLE_AGE", "WAS_CLAIM_PAID",
    "PREVIOUS_CLAIM_PAID", "PREVIOUS_POLICYHOLDERS"
]].copy()

X["INSURED_VALUE"] = np.log1p(X["INSURED_VALUE"] + 1)

X = pd.get_dummies(
    X,
    columns=["SEX", "INSR_TYPE", "INSR_COVER", "TYPE_VEHICLE", "WAS_CLAIM_PAID"],
    drop_first=True
)

Y = insurance_data["LOG_PREMIUM"]

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=87
)

model3 = XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=87,
    n_jobs=-1
)

model3.fit(X_train, Y_train)

Y_pred = model3.predict(X_test)

r_sq = r2_score(Y_test, Y_pred)
mae = mean_absolute_error(Y_test, Y_pred)

print(f"R²: {r_sq:.4f}")
print(f"MAE: {mae:.2f}")

plot_importance(model3, max_num_features=20, importance_type='weight')
plt.show()

After feature selection, we are left with 10 meaningful variables. Some variables that were quite good predictors, such as insurance start day, were not keep as they would be known to the insurer before. Other variables, such as previous policy holders and previous claim paid, that were feature-enginnered proved extemely valuable. Also, the log-transformation of premiums also improved model performance.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

axes[0].scatter(Y_test, Y_pred, alpha=0.05)
axes[0].plot([Y_test.min(), Y_test.max()],
             [Y_test.min(), Y_test.max()],
             "r")
axes[0].set_title("Accuracy Plot")
axes[0].set_xlabel("Actual")
axes[0].set_ylabel("Predicted")

axes[1].scatter(Y_pred, Y_test - Y_pred, alpha=0.05)
axes[1].axhline(y=0, color="r")
axes[1].set_xlabel("Predicted")
axes[1].set_ylabel("Residuals")
axes[1].set_title("Residual Plot")

axes[2].hist(Y_test - Y_pred, bins=30, density=True)
axes[2].set_xlabel("Residuals")
axes[2].set_ylabel("Frequency")
axes[2].set_title("Distribution of Residuals")

plt.tight_layout()
plt.show()

As we can see from the accuracy plot, the data points have enveloped the accuracy line, with the same being in the residual plot. The distribution of residuals have also taken a more normal-looking distribution with the spike at 0 being less pronounced. The lines that appear in the accuracy & residuals plots are most likely caused by rounded insurance premium values in the original dataset.

#### Bayesian optimization

One other way that could improve the machine learning model is bayesian optimization. Instead of going through all available parameter combinations, we can use stochastic methods to look for the minimum in our loss function. For that, we'll use the hyperopt library functionalities.

In [ ]:
model = XGBRegressor(
    n_jobs=-1,
    random_state=87,
    tree_method='hist',
    device='cuda'
)

search_space = {
    'n_estimators': Integer(100, 1000),
    'max_depth': Integer(3, 12),
    'learning_rate': Real(0.005, 0.2, prior='log-uniform'),
    'subsample': Real(0.6, 1.0, prior='uniform'),
    'colsample_bytree': Real(0.6, 1.0, prior='uniform'),
    'gamma': Real(0, 5, prior='uniform'),
    'min_child_weight': Integer(1, 10)
}

optimizer = BayesSearchCV(
    estimator=model,
    search_spaces=search_space,
    n_iter=100,
    cv=3,
    scoring='neg_mean_absolute_error',
    n_jobs=1,
    random_state=87
)

optimizer.fit(X_train, Y_train)

print(f"Best parameters found: {optimizer.best_params_}")
print(f"Best CV score (MAE): {-optimizer.best_score_:.4f}")

Y_pred = optimizer.predict(X_test)

r_sq = r2_score(Y_test, Y_pred)
mae = mean_absolute_error(Y_test, Y_pred)

print(f"\nFinal R² on Test Set: {r_sq:.4f}")
print(f"Final MAE on Test Set: {mae:.2f}")


final_model = optimizer.best_estimator_
plot_importance(final_model, max_num_features=20, importance_type='weight')
plt.title("XGBoost Feature Importance (Best Model)")
plt.show()

After testing 100 combinations of variables the minimum of loss function of 0.27 was achieved by looking for the lowest mean absolute error. With the optimal hyperparamets, the model achieved an R<sup>2</sup> of 0.9058, an improvement of 0.0124 over the previous model.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

axes[0].scatter(Y_test, Y_pred, alpha=0.1)
axes[0].plot([Y_test.min(), Y_test.max()],
             [Y_test.min(), Y_test.max()],
             "r")
axes[0].set_title("Accuracy Plot")
axes[0].set_xlabel("Actual")
axes[0].set_ylabel("Predicted")

axes[1].scatter(Y_pred, Y_test - Y_pred, alpha=0.1)
axes[1].axhline(y=0, color="r")
axes[1].set_xlabel("Predicted")
axes[1].set_ylabel("Residuals")
axes[1].set_title("Residual Plot")

axes[2].hist(Y_test - Y_pred, bins=30, density=True)
axes[2].set_xlabel("Residuals")
axes[2].set_ylabel("Frequency")
axes[2].set_title("Distribution of Residuals")

plt.tight_layout()
plt.show()